<H1> The ETL for The DataWareHouse</H

In [23]:
import yfinance as yf #the API used to extract data from the internet
import pandas as pd

<H2>Exracting data for Apple company </H2>

In [24]:
# Download AAPL stock data for the last 10 years (you can adjust the period)
aapl_data = yf.download("AAPL", start="2013-12-31", end="2024-01-01")
aapl_data.to_csv('AAPL_stock_data.csv') # Save the data to a CSV file

# Download AAPL dividends data
aapl_dividends = yf.Ticker("AAPL").dividends
aapl_dividends = aapl_dividends[aapl_dividends.index >= '2014-01-01'] 
aapl_dividends.to_csv('AAPL_dividends.csv')

[*********************100%***********************]  1 of 1 completed


In [25]:
prices = pd.read_csv('AAPL_stock_data.csv') # Load the stock price data (AAPL)
dividends = pd.read_csv('AAPL_dividends.csv') # Load the dividends data (AAPL)


<H3>Exploring the data in our data set </H3>

In [26]:
print("The Data Types inside prices \n",prices.dtypes) #prints the datatype of every coulmn 
print("The Data Types inside dividends")
print(dividends.dtypes)

#check for nulls, duplicates  
print(prices.isnull().sum())
print(dividends.isnull().sum())

print(prices.duplicated().sum())
print(dividends.duplicated().sum())


The Data Types inside prices 
 Price        object
Adj Close    object
Close        object
High         object
Low          object
Open         object
Volume       object
dtype: object
The Data Types inside dividends
Date          object
Dividends    float64
dtype: object
Price        0
Adj Close    1
Close        1
High         1
Low          1
Open         1
Volume       1
dtype: int64
Date         0
Dividends    0
dtype: int64
0
0


<H2> Transforming Data </H2>

In [27]:
# Drop the first two rows from the prices and divedins dataset
prices = prices.drop([0, 1])
dividends = dividends.drop([0, 1])

# Reset the index of both datasets and rename wht we need 
prices.reset_index(drop=True, inplace=True)
dividends.reset_index(drop=True, inplace=True)
prices.rename(columns={'Price' : 'Date'}, inplace=True)
prices.rename(columns={'Adj Close' : 'Adj_Close'}, inplace=True)

# Convert the 'Date' column to a valid MySQL-compatible date format
prices['Date'] = pd.to_datetime(prices['Date'], errors='coerce')  # Convert to datetime
prices['Date'] = prices['Date'].dt.strftime('%Y-%m-%d')  # Format as 'YYYY-MM-DD'

# Check for invalid dates and remove them if necessary
if prices['Date'].isnull().any():
    print("Found invalid dates. Dropping them...")
    prices = prices.dropna(subset=['Date'])


<H2> Load data into csv To feed them in MySQL datawarehouse</H2>

In [28]:
dividends.to_csv('AAPL_dividends_cleaned.csv', index=False)
prices.to_csv('AAPL_stock_data_cleaned.csv', index=False)